# Import Packages

In [1]:
from decouple import config
import time
import re

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

import google.generativeai as genai
from google.api_core.exceptions import InternalServerError

API_KEY = config("API_KEY")
genai.configure(api_key=API_KEY)

/Users/johnny_hsieh/anaconda3/envs/rag_gemini/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set parameters

In [2]:
# Sentence embedding
EMBED_MODEL_NAME = "DMetaSoul/sbert-chinese-general-v2"

# Gemini Settings
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

generation_config = {
  "temperature": 0,
  "max_output_tokens": 512,
  "response_mime_type": "text/plain",
}

sort_model = genai.GenerativeModel(
    "gemini-1.5-flash", 
    safety_settings, 
    generation_config=generation_config
    )

gen_ans_model = genai.GenerativeModel(
    "gemini-pro", 
    safety_settings, 
    generation_config=generation_config
    )

# Function for loading vector DB and generating a retriever

In [3]:
def generate_retriever():
    print("Loading vector DB...")
    model_kwargs = {"device": "mps"}
    embedding = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME, model_kwargs=model_kwargs)
    
    db = FAISS.load_local("../diabetic_qa_vector_db", embedding, allow_dangerous_deserialization=True)
    print("Done loading vector DB!\n")
    return db.as_retriever(search_type="mmr", search_kwargs={"k": 3})

# Function for searching the related content

In [4]:
def search_related_content(retriever, query):
    docs = retriever.invoke(query)
    related_context = "\n---\n".join([doc.page_content for doc in docs])
    return related_context

# Function for generating the answer

In [5]:
def generate_answer(query, sort_result):
    pattern = r'[\n*]'
    template = f"""
任務: 
1. 你是一位在台灣的糖尿病領域的專業護理師，需要以親切的口吻回答病患的問題。
2. 你必須依照下方的「相關文本」，再透過同義的改寫生成出最後的答案。
3. 輸出限制： 最多60字、只能使用繁體中文、純文字格式
4. 如果「相關文本」中有答案，一定要引用「相關文本」來回答；如果判斷與「病患的提問」沒有關連只要回答「不好意思，我不清楚。」即可。
------
「相關文本」：
{sort_result}
------
「病患的提問」：
{query}
"""
    print(template)
    start_time = time.time()
    response = gen_ans_model.generate_content(template)
    qa_result = response.text
    qa_result = re.sub(pattern, "", qa_result)
    end_time = time.time()
    print(f"Generate Answer Time: {end_time - start_time}")
    return qa_result

# Function for sorting the similarity of the related content

In [6]:
def sort_generate_answer(query, related_context):
    pattern = r'[#\n*]'
    template = f"""
你是一位在台灣的糖尿病領域的專業護理師。
任務: 
1. 比較「相關文本」中三個段落的文字與「病患的提問」，對「相關文本」中三個段落的文字進行相關性的排序，相關性由高到低排序。
2. 輸出限制：段落文字(相關性最高)即可
3. 如果「相關文本」中有答案，一定要引用「相關文本」來回答；如果判斷與「病患的提問」沒有關連只要回答「不好意思，我不清楚。」即可。
------
「相關文本」：
{related_context}
------
「病患的提問」：
{query}
"""
    print(template)
    start_time = time.time()
    try:
        response = sort_model.generate_content(template)
        sort_result = response.text
        sort_result = re.sub(pattern, "", sort_result)
        print(sort_result)
        end_time = time.time()
        print(f"Sort Time: {end_time - start_time}")
        qa_result = generate_answer(query, sort_result)
        return qa_result
    except InternalServerError as e:
        return "不好意思，請您稍等一分鐘再重新提問，請您包涵，謝謝！(Google忙線中)"

# Generate retriever

In [7]:
retriever = generate_retriever()

Loading vector DB...


/Users/johnny_hsieh/anaconda3/envs/rag_gemini/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Done loading vector DB!



# Search related content, sorting and generate answer

In [8]:
question = [
    "為什麼糖尿病患者不敢吃水果？",
    "糖尿病患者該怎麼吃水果比較好？",
    "每天建議吃多少水果？",
    "水果的升糖指數會被哪些因素影響？",
    "一份水果大概是多少？",
    "紅豆是什麼種類的食物？",
    "喝無糖紅豆湯會對血糖有什麼影響？",
    "什麼情況下建議患者改用胰島素？",
    "哪些糖尿病患者需要直接打胰島素？",
    "打胰島素會不會導致需要洗腎？",
    "有哪些方法可以更完整地記錄飲食？",
    "記錄飲食時哪些資訊最重要？",
    "為什麼記錄飲食時，醣量是關鍵指標？",
    "熱量對於哪些人來說是關鍵的指標？",
    "記錄飲食時容易遇到哪些問題？",
    "烹調方式會怎麼影響飲食控制？",
    "氣喘如何治療？"
]

answers = []
for query in question:
    related_context = search_related_content(retriever, query)
    result = sort_generate_answer(query, related_context)
    print(result)
    answers.append(result)
    print("-" * 30)
    time.sleep(5)


你是一位在台灣的糖尿病領域的專業護理師。
任務: 
1. 比較「相關文本」中三個段落的文字與「病患的提問」，對「相關文本」中三個段落的文字進行相關性的排序，相關性由高到低排序。
2. 輸出限制：段落文字(相關性最高)即可
3. 如果「相關文本」中有答案，一定要引用「相關文本」來回答；如果判斷與「病患的提問」沒有關連只要回答「不好意思，我不清楚。」即可。
------
「相關文本」：
糖尿病友為什麼不敢吃⽔果？因為怕⾎糖⾼。
---
⽔果的升糖指數受哪些因素影響？澱粉組成、含醣總量、加⼯和烹調⽅式、膳食纖維質及種類。
---
建議糖尿病友如何食用⽔果？建議直接食用原型⽔果，避免果汁、加⼯⽔果及果醬。
------
「病患的提問」：
為什麼糖尿病患者不敢吃水果？

糖尿病友為什麼不敢吃⽔果？因為怕⾎糖⾼。 
Sort Time: 3.8545262813568115

任務: 
1. 你是一位在台灣的糖尿病領域的專業護理師，需要以親切的口吻回答病患的問題。
2. 你必須依照下方的「相關文本」，再透過同義的改寫生成出最後的答案。
3. 輸出限制： 最多60字、只能使用繁體中文、純文字格式
4. 如果「相關文本」中有答案，一定要引用「相關文本」來回答；如果判斷與「病患的提問」沒有關連只要回答「不好意思，我不清楚。」即可。
------
「相關文本」：
糖尿病友為什麼不敢吃⽔果？因為怕⾎糖⾼。 
------
「病患的提問」：
為什麼糖尿病患者不敢吃水果？

Generate Answer Time: 3.3793630599975586
糖尿病友怕吃水果是因為擔心血糖升高。
------------------------------

你是一位在台灣的糖尿病領域的專業護理師。
任務: 
1. 比較「相關文本」中三個段落的文字與「病患的提問」，對「相關文本」中三個段落的文字進行相關性的排序，相關性由高到低排序。
2. 輸出限制：段落文字(相關性最高)即可
3. 如果「相關文本」中有答案，一定要引用「相關文本」來回答；如果判斷與「病患的提問」沒有關連只要回答「不好意思，我不清楚。」即可。
------
「相關文本」：
建議糖尿病友如何食用⽔果？建議直接食用原型⽔果，避免果汁、加⼯⽔果及果醬。
---
⽔果的升糖指數受哪些因素影響？澱粉組成、含醣總量、加⼯和烹調⽅式、膳食纖維

In [9]:
for q, a in zip(question,answers):
    print(q, a)

為什麼糖尿病患者不敢吃水果？ 糖尿病友怕吃水果是因為擔心血糖升高。
糖尿病患者該怎麼吃水果比較好？ 建議直接吃新鮮水果，避免果汁、加工水果和果醬。
每天建議吃多少水果？ 每天建議攝取 2 份水果。
水果的升糖指數會被哪些因素影響？ 水果的升糖指數會受澱粉組成、含醣總量、加工和烹調方式、膳食纖維質及種類等因素影響。
一份水果大概是多少？ 一份水果大約像棒球大小，或是一般碗切好在碗裡約 8 分滿。
紅豆是什麼種類的食物？ 紅豆屬於全穀雜糧類食物。
喝無糖紅豆湯會對血糖有什麼影響？ 無糖紅豆湯會影響血糖。
什麼情況下建議患者改用胰島素？ 當其他降血糖藥物無法穩定控制血糖時，建議改用胰島素。
哪些糖尿病患者需要直接打胰島素？ 有些糖尿病初期患者需要直接打胰島素。
打胰島素會不會導致需要洗腎？ 胰島素本身不會導致洗腎。
有哪些方法可以更完整地記錄飲食？ 使用文字、拍照或食物資料庫等方法。
記錄飲食時哪些資訊最重要？ 記錄飲食時最重要的資訊是醣量和熱量。
為什麼記錄飲食時，醣量是關鍵指標？ 因為醣量會直接影響飯後血糖。
熱量對於哪些人來說是關鍵的指標？ 對於有體重管理需求的人來說，熱量是關鍵的指標。
記錄飲食時容易遇到哪些問題？ 記錄飲食時，容易只寫食物名稱而不記錄份量或烹調方式。
烹調方式會怎麼影響飲食控制？ 烹調方式會影響食物熱量，進而影響飲食控制。
氣喘如何治療？ 不好意思，我不清楚。
